## Set up

#### Import Libraries

In [14]:
from utils import utils



In [15]:
# TO DO: check if lines 8-11 are needed 

RANDOM_SEED = 6
np.random.seed(RANDOM_SEED)

# torch.backends.cudnn.benchmark = False # uses deterministic convolution algorithm (may reduce performance)
# torch.backends.cudnn.deterministic = True #

#Sets the seed of RNG (GPU and CPU)
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

torch.cuda.manual_seed_all(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED) 

In [16]:
# Set settings
features, input_window, output_window, stride = set_settings()

print(f'features: {features}')
print(f'input_window: {input_window}')
print(f'output_window: {output_window}')
print(f'stride: {stride}')

features: ['Hips Flexion-Extension Left', 'Knees Flexion-Extension Left', 'Ankles Dorsiflexion-Plantarflexion Left', 'Hips Flexion-Extension Right', 'Knees Flexion-Extension Right', 'Ankles Dorsiflexion-Plantarflexion Right']
input_window: 100
output_window: 1
stride: 1


In [4]:
# Set device
print(f'The device used in this notebook is: {setDevice()}')

DEVICE = setDevice()

The device used in this notebook is: cuda


#### Load Data 

In [5]:
file_dir = r'D:\Study 2 Data\Healthy Gait\Train - Copy'
train_files = os.listdir(file_dir) 

# Changes the working directory to get the data from their location 
os.chdir(file_dir)
cwd = os.getcwd()

print(f'Current working directory is: {cwd}')
print(f"There are {len(train_files)} files in the specified path.")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\Study 2 Data\\Healthy Gait\\Train - Copy'

In [ ]:
train_files

In [ ]:
#Create feature list to extract values needed from CSV files
all_features = ['Trial', 'Time'] + features
all_features

#### Extract Data from CSV into Pandas Dataframe

In [ ]:
all_data = create_dataframe(train_files, all_features)

In [ ]:
all_data.reset_index(drop=True, inplace=True) #reset the index of the table
# path = r'D:\Study 2 Data\Healthy Gait' + '\\' + 'all_data_healthy_train.csv'
# all_data.to_csv(path)

In [ ]:
#export data

In [ ]:
n_trials = all_data['Trial'].max()
print(f'maximum number of trials is: {n_trials}')

In [ ]:
all_data['Patient ID'].unique()

#### Divide into training and validation sets

In [ ]:
train_subjects = ['AB3154 BF T6-10', 
                     'AB6751 BF T1-5', 
                     'AB7779 BF T1-5',
                     'AB9737 BF T1-5', 
                     'AB9737 BF T6-10',
                     'AB7422 BF T1-5',
                     ]

val_subjects = ['AB9119 BF T1-5']

In [ ]:
train_data = all_data.loc[all_data['Patient ID'].isin(train_subjects)]

val_data = all_data.loc[all_data['Patient ID'].isin(val_subjects)]

In [ ]:
print(f'all_data: {all_data.shape}')
print(f'train_data: {train_data.shape}')
print(f'val_data: {val_data.shape}')

## Data Analysis

#### Data Analysis

In [ ]:
all_data.isnull().sum()

In [ ]:
ax = sns.violinplot(x="Patient ID", y="Hips Flexion-Extension Left", 
                    data=all_data, palette="muted")

In [ ]:
# ax = sns.violinplot(x="Trial", y="Hips Flexion-Extension Right", 
#                     data=all_data[all_data['Patient ID'] == "AB3154 T6-10 BF"], palette="muted")

In [ ]:
# ax = sns.violinplot(x="Trial", y="Hips Flexion-Extension Right", 
#                     data=all_data[all_data['Patient ID'] == "AB5498 BF T6-10"], palette="muted")

## Data Processing

In [ ]:
train_samples = count_nsamples(train_data)
val_samples = count_nsamples(val_data)

In [ ]:
print(f'Number of train samples: {train_samples}')
print(f'Number of validation samples: {val_samples}')

In [ ]:
# all_data['Patient ID'] == "AB5498 BF T6-10" 

In [ ]:
# os.makedirs('D:\Study 2 Data\Pre-process sample', exist_ok=True)  #check if directory exists
# all_data.to_csv('D:\Study 2 Data\Pre-process sample/exported-data3.csv')  #export data to csv

#### Convert Data into numpy array for forming windows

In [ ]:
m = all_data['Patient ID'].unique()

In [ ]:

def pd_to_np_converter(data, n_samples, features):
    #create a numpy array that stores the data for export
    sample_ID = []
    # patients = 2
    # n_trials = 10
    # # samples = patients * n_trials
    data_store = np.zeros((n_samples, 2000, len(features)), dtype=np.float32)
    i = 0

    for p in data['Patient ID'].unique(): #loop over patients 
        for t in data['Trial'].unique(): #loop over trials starting with trials 1 to trial 9 (inclusive)
            pd_array = data[(data['Patient ID'] == p) & (data['Trial'] == t)]
            if pd_array.empty:
                continue
                # print('DataFrame is empty!')
                # print(f'Trail {t} does not exist in {p}')
            else:
                np_array = pd_array.to_numpy()
                data_store[i, :np_array.shape[0], :] = np_array[:,3:] 
                sample_ID.append(p+ ' Ts'+str(t)) 
                i +=1

    return pd_array.columns, data_store


In [ ]:
train_columns, train_data_np = pd_to_np_converter(train_data, train_samples, features)
val_columns, val_data_np = pd_to_np_converter(val_data, val_samples, features)

print(f'train_data_np.shape: {train_data_np.shape}')
print(f'val_data_np.shape: {val_data_np.shape}')

In [ ]:
train_columns[3:].tolist() 

In [ ]:
features == train_columns[3:].tolist() 

In [ ]:
labels_keys = train_columns[3:].tolist() #copy the train columns removing the first column headers'Patient ID', 'Trial', 'Time'


if features == labels_keys: # check that the features are the same as the label keys 
    print('YAY! Column headers of dataframe match features')
else:
    print('ERROR: Features and labels_keys do not match!')

#### Generate Dictionary for Features

In [ ]:
labels_keys = features 

labels_idx = np.arange(0, len(labels_keys), 1)

labels = dict(zip(labels_keys, labels_idx))

len(labels)
labels
# labels_keys

#### Process Data: Window Generation

In [ ]:
# Creating training datasets
# Selecting the features to be used when creating windows 
approx_seq_len = 2000 # appoximate the length of the longest sequence that can be encountered 
# samples_per_file = ((approx_seq_len - (input_window+output_window)) // stride) + 1 #number of window samples generated per file 
samples_per_file = 800


# create a zero-filled 3D array with shape (number of samples * number of files, window size, number of features)
X_train_windows = np.zeros((samples_per_file*train_samples, input_window, len(features)), dtype=np.float32) #size can be reduced by decreasing train_size 
Y_train_windows = np.zeros((samples_per_file*train_samples, output_window, len(features)), dtype=np.float32) 


start_idx = 0 #setting start index to equal zero 
train_sample_sum = 0
train_excluded_samples = []
# Create training windows 

#for i in tqdm(range(train_size)): #Use for including all data including outliers 
for i in range(train_samples): 
       
    X_values, Y_values = window_generator(
        train_data_np[i,:,:],
        input_window=input_window, 
        output_window=output_window, 
        stride=stride, 
        features=features,
        labels=labels
        )

    end_idx = start_idx + X_values.shape[0]

    # print(f'file number [{f}] start index: {start_idx}, end index: {end_idx}, number of samples: {X_values.shape[0]}')

    X_train_windows[start_idx:end_idx, :, :] = X_values
    Y_train_windows[start_idx:end_idx, :, :] = Y_values

    # print(f'number of samples copied: {X_train_data_store[start_idx:end_idx, :, :].shape[0]}')

    start_idx = end_idx 
    train_sample_sum += X_values.shape[0]

    # except Exception:
    #     exception_msg(i)
    #     train_excluded_samples.append(i)

# print(f"Completed storage of training windows samples, which contains {X_train_data_store.shape[0]} samples")
X_train_data = X_train_windows[:end_idx, :, :]
Y_train_data = Y_train_windows[:end_idx, :, :]


print(f'shape of X_train_windows: {X_train_windows.shape}')
print(f'shape of Y_train_windows: {Y_train_windows.shape}')

print(f'shape of X_train_data: {X_train_data.shape}')
print(f'shape of Y_train_data: {Y_train_data.shape}')

In [ ]:
# Creating validation datasets
# Selecting the features to be used when creating windows 
approx_seq_len = 2000 # appoximate the length of the longest sequence that can be encountered 
# samples_per_file = ((approx_seq_len - (input_window+output_window)) // stride) + 1 #number of window samples generated per file 
samples_per_file = 800


# create a zero-filled 3D array with shape (number of samples * number of files, window size, number of features)
X_val_windows = np.zeros((samples_per_file*val_samples, input_window, len(features)), dtype=np.float32) #size can be reduced by decreasing train_size 
Y_val_windows = np.zeros((samples_per_file*val_samples, output_window, len(features)), dtype=np.float32) 


start_idx = 0 #setting start index to equal zero 
val_sample_sum = 0
val_excluded_samples = []
# Create training windows 

#for i in tqdm(range(train_size)): #Use for including all data including outliers 
for i in range(val_samples): 
       
    X_values, Y_values = window_generator(
        val_data_np[i,:,:],
        input_window=input_window, 
        output_window=output_window, 
        stride=stride, 
        features=features,
        labels=labels
        )

    end_idx = start_idx + X_values.shape[0]

    # print(f'file number [{f}] start index: {start_idx}, end index: {end_idx}, number of samples: {X_values.shape[0]}')

    X_val_windows[start_idx:end_idx, :, :] = X_values
    Y_val_windows[start_idx:end_idx, :, :] = Y_values

    # print(f'number of samples copied: {X_train_data_store[start_idx:end_idx, :, :].shape[0]}')

    start_idx = end_idx 
    val_sample_sum += X_values.shape[0]

    # except Exception:
    #     exception_msg(i)
    #     train_excluded_samples.append(i)

# print(f"Completed storage of training windows samples, which contains {X_train_data_store.shape[0]} samples")
X_val_data = X_val_windows[:end_idx, :, :]
Y_val_data = Y_val_windows[:end_idx, :, :]


print(f'shape of X_val_windows: {X_val_windows.shape}')
print(f'shape of Y_val_windows: {Y_val_windows.shape}')

print(f'shape of X_val_data: {X_val_data.shape}')
print(f'shape of Y_val_data: {Y_val_data.shape}')

In [ ]:
# Plot histogram of the data 
for f in range(len(features)):
    plt.hist(Y_train_data[:,:,f].reshape(-1,1), label = features[f], bins=50, range=(Y_train_data.min(), Y_train_data.max()))
    plt.legend()
    plt.show()

#### Process Data: Normalisation/Standarisation

In [ ]:
X_train_norm, scalars = normalise_fit(X_train_data)
Y_train_norm = normalise_transform(Y_train_data, scalars)


X_val_norm = normalise_transform(X_val_data, scalars)
Y_val_norm = normalise_transform(Y_val_data, scalars)


scalars

In [ ]:
# Plot histogram of the data 
for f in range(len(features)):
    plt.hist(Y_train_norm[:,:,f].reshape(-1,1), bins=50, label = features[f])
    plt.legend()
    plt.show()

In [ ]:
# Convert to Tensor 
# do not store on GPU (yet)
X_train = torch.from_numpy(X_train_norm).float()
Y_train = torch.from_numpy(Y_train_norm).float()

X_val = torch.from_numpy(X_val_norm).float()
Y_val = torch.from_numpy(Y_val_norm).float()

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

print(f'X_val shape: {X_val.shape}')
print(f'Y_val shape: {Y_val.shape}')


In [ ]:
# # Creates dataset object that gets individual samples for training/testing so that the Dataloader can generate batches
# class gaitDataset(Dataset):
#     def __init__(self, x, y):
#         self.x = x 
#         self.y = y 

#     def __len__(self):
#         return self.x.shape[0]
    
#     def __getitem__(self, index):
#         X_sample = self.x[index, :, :]
#         # decoder_sample = self.y[index,:-1,:]
#         Y_sample = self.y[index, 1:, :]
#         return X_sample, Y_sample

In [ ]:
train_dataset = gaitDataset(X_train, Y_train)
# train_dataloader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle = False)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle = True)

val_dataset = gaitDataset(X_val, Y_val) #ADJUSTED
# val_dataloader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)


print(f"Train Dataset length: {len(train_dataset)}")
print(f"Val Dataset length: {len(val_dataset)}")


## Training Models

#### Deep Learning Models

In [ ]:
# # useful resources: https://www.youtube.com/watch?v=8A6TEjG2DNw (LSTM Time Series Prediction Tutorial using PyTorch in Python | Coronavirus Daily Cases Forecasting)
# LSTM model 
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, in_seq_len, out_seq_len, output_size, device):
        super(LSTM, self).__init__()
        '''
        nn.lstm: 
        input_size = number of features in input, if batch_first = TRUE (batchSize, seq_len, Hin) FEATURES
        hidden_size = number of features in hidden state
        num_layers
        batch_first = if TRUE (batch, seq, Hin) Hin may mean input features
        h_0 = (D * num_layers, batchSize, Hout)
        c_0 = (D * num_layers, batchSize, Hcell)

        output = (batchSize, seq_len, D * Hout) Hout may mean output features, if batch first = TRUE 
        h_0 = (D*num_layers, N, Hout) final state for each element in the batch 
        C_0 = (D*num_layers, N, Hcell) final cell state for each element 

        nn.linear:
        input_size = (N,*, Hin) Hin = input_features (equal to hidden size therefore is arbitrarily set)
        output_size = (N, *, Hout) Hout = output features (equal to output_size passed in to the model) 

        '''
        # Pytorch documentation: 
        # >>> rnn = nn.LSTM(10, 20, 2) features, hidden_size, number of layers 
        # >>> input = torch.randn(5, 3, 10) in_seq_len, batch_size, hidden_size (batch size should not be input to the mode, inferred from the shape of the input and will be the first number if batch_first = TRUE)
        # >>> h0 = torch.randn(2, 3, 20)
        # >>> c0 = torch.randn(2, 3, 20)
        # >>> output, (hn, cn) = rnn(input, (h0, c0))

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.in_seq_len = in_seq_len
        self.output_size = output_size
        self.out_seq_len = out_seq_len
        self.device = device
        
        
        # nn.LSTM(features, hidden_size, number of layers)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first = True)

        #nn.fc1
        self.fc1 = nn.Linear(in_features=self.hidden_size, out_features=(self.output_size * self.out_seq_len))
        # self.fc1 = nn.Linear(in_features=self.hidden_size, out_features=(self.output_size * self.out_seq_len))


    def forward(self, input_data):

        # initialise hidden and cell states after passing through each batch (this is skipped in stateless model)
        h_0 = torch.zeros(self.num_layers, input_data.shape[0], self.hidden_size).to(self.device) # input_data.shape[0] is equal to batch size
        c_0 = torch.zeros(self.num_layers, input_data.shape[0], self.hidden_size).to(self.device)

        #propagate through LSTM
        lstm_out, (h_out, c_out) = self.lstm(input_data, (h_0, c_0))
        # lsmt_out.shape = (batch_size,seq_length, hidden_size)
        # print('lstm_out[-1][-1]')
        # print(lstm_out[-1][-1])

        # print('h_out[-1][-1]')
        # print(h_out[-1][-1])
        # if lstm_out[-1][-1] == h_out[-1][-1]:
        #     print(lstm_out[-1][-1])
            
        # print(f'lsmt_out: {lstm_out.shape}')
        # print(f'h_out: {h_out.shape}')
        # print(f'h_out: {h_out.shape}')

        # propagate through linear layer 
        fc1_out = self.fc1(h_out[-1])
        # fc1_out = self.fc1(lstm_out)
        # print(f'preds.shape (before reshaping): {fc1_out.shape}')
        
        preds = fc1_out.reshape(input_data.shape[0], self.out_seq_len, self.output_size)
        # print(f'preds.shape (after reshaping): {preds.shape}')
        
        return preds

In [ ]:
### positional encoder: https://pytorch.org/tutorials/beginner/transformer_tutorial.html 

# the points is to generate a matrix, swize [max_len, 1, d_model]
import math

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1) #shape would be [max_len, 1]

        #an array with shape [d_model size /2] since we are only taking even numbers is multiplied by a constant. -math log(10000) is equal -9.2103. multiplying the array by a negative constant means all the numbers in the array are negative     
        #get exp term for each element in the array #
        #we divide d_model by two because we will multiply by cosine and sine so we will retain back the d_model shape in the end                                                          

        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)) 

        pe = torch.zeros(max_len, 1, d_model) #shape [max_len, 1 , d_model]
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe) #we do not need to store the gradients of the positional encoder since they will not change 

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        #add positional encoding layer to input and apply dropout. Each timestep will be multiplied by a different positional encoder values
        x = x + self.pe[:x.size(0)]
        # print(f'x.shape {x.shape}')
        # return self.dropout(x)
        return x


In [ ]:
#Source: https://github.com/windofshadow/THAT/blob/1e35c0713ee7d4bc0494f46c3fa118bf406d9c58/TransCNN.py 

def normal_pdf(pos, mu, sigma):
    a = pos - mu
    log_p = -1*torch.mul(a, a)/(2*sigma) - torch.log(sigma)/2
    return F.softmax(log_p, dim=1)

In [ ]:
#Source: https://github.com/windofshadow/THAT/blob/1e35c0713ee7d4bc0494f46c3fa118bf406d9c58/TransCNN.py
class Gaussian_Position(nn.Module):
    def __init__(self, d_model, total_size, K=10):
        super(Gaussian_Position, self).__init__()
        #self.embedding = get_pe(d_model, K).to('cuda')
        #self.register_buffer('pe', self.embedding)
        self.embedding = nn.Parameter(torch.zeros([K, d_model], dtype=torch.float), requires_grad=True)
        nn.init.xavier_uniform_(self.embedding, gain=1)
        self.positions = torch.tensor([i for i in range(total_size)], requires_grad=False).unsqueeze(1).repeat(1, K).to('cuda')
        s = 0.0
        interval = total_size / K
        mu = []
        for _ in range(K):
            mu.append(nn.Parameter(torch.tensor(s, dtype=torch.float), requires_grad=True))
            s = s + interval
        self.mu = nn.Parameter(torch.tensor(mu, dtype=torch.float).unsqueeze(0), requires_grad=True)
        self.sigma = nn.Parameter(torch.tensor([torch.tensor([50.0], dtype=torch.float, requires_grad=True) for _ in range(K)]).unsqueeze(0))

    def forward(self, x):
        M = normal_pdf(self.positions, self.mu, self.sigma)
        pos_enc = torch.matmul(M, self.embedding)
        #print(M)
        return x + pos_enc.unsqueeze(0).repeat(x.size(0), 1, 1)

In [ ]:
# # dropout = nn.Dropout(p=dropout)

# max_len = 5
# d_model=4
# position = torch.arange(max_len).unsqueeze(1) #shape would be [max_len, 1]

# #an array with shape [d_model size /2] since we are only taking even numbers is multiplied by a constant. -math log(10000) is equal -9.2103. multiplying the array by a negative constant means all the numbers in the array are negative     
# #get exp term for each element in the array #
# #we divide d_model by two because we will multiply by cosine and sine so we will retain back the d_model shape in the end                                                          

# div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)) 

# pe = torch.zeros(max_len, 1, d_model) #shape [max_len, 1 , d_model]
# pe[:, 0, 0::2] = torch.sin(position * div_term)
# pe[:, 0, 1::2] = torch.cos(position * div_term)
# # self.register_buffer('pe', pe) #we do not need to store the gradients of the positional encoder since they will not change 



# # def forward(self, x):
# #     """
# #     Args:
# #         x: Tensor, shape [seq_len, batch_size, embedding_dim]
# #     """
# #     x = x + self.pe[:x.size(0)]
# #     print(f'x.shape {x.shape}')
# #     return self.dropout(x)

# pe

In [ ]:
# arry = torch.zeros((5, 3, 4))
# arry

In [ ]:
# pe[:arry.size(0)]

In [ ]:
# arry = arry + pe[:arry.size(0)]
# # 
# arry

In [ ]:
# https://towardsdatascience.com/how-to-make-a-pytorch-transformer-for-time-series-forecasting-69e073d4061e

# https://github.com/Doheon/TimeSeriesForecast-Transformer/blob/main/TransformerTimeSeries.ipynb

import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset


class TransformerModel(nn.Module):
    def __init__(self, input_size: int,
                    in_seq_len: int,  
                    max_seq_len: int,
                    out_seq_len: int,
                    d_model: int=512, # aka d_model 
                    n_encoder_layers: int=4,
                    n_decoder_layers: int=4,
                    n_heads: int=8,
                    dropout_encoder: float=0.2, 
                    dropout_decoder: float=0.2,
                    dim_feedforward_encoder: int=2048, #2048 is default value
                    linear_layer1_dim: int=200,
                    dim_feedforward_decoder: int=200
                    ): 

        super().__init__()

        # self.dec_seq_len = dec_seq_len


        # self.in_seq_len = in_seq_len
        # self.output_size = input_size
        # self.out_seq_len = out_seq_len
        # self.model_type = 'Transformer'

        self.encoder_input_layer = nn.Linear( #incput to encoder is linear layer
            in_features=input_size, 
            out_features=d_model
            )
     

        # self.pos_encoder = PositionalEncoding(
        #     d_model = d_model, 
        #     dropout = 0.5, 
        #     max_len = max_seq_len)

        self.pos_encoder =  Gaussian_Position(
            d_model = d_model, 
            total_size = max_seq_len)

        #encoder layer is made of self attention and feedforward network (aka tiny transformer block)
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=n_heads, 
            dim_feedforward=dim_feedforward_encoder, 
            dropout=dropout_encoder,
            batch_first=True)


        #pass the architecture of a single encoder layer designed and specify how many encoder layer put after each other
        self.encoder = nn.TransformerEncoder(
            encoder_layer=encoder_layers,
            num_layers=n_encoder_layers, 
            norm=None
            )
        
        self.decoder_input_layer = nn.Linear(
            in_features=input_size, 
            out_features=d_model
            ) 
        
        
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=d_model, 
            nhead=n_heads,
            dim_feedforward=dim_feedforward_decoder,
            dropout=dropout_decoder,
            batch_first=True
            )

        self.decoder = nn.TransformerDecoder(
            decoder_layer=decoder_layer,
            num_layers=n_decoder_layers, 
            norm=None
            )

        self.linear_layer1 = nn.Linear(
            in_features=3*d_model,   #
            out_features=linear_layer1_dim
            )

        self.linear_layer2 = nn.Linear(
            in_features=linear_layer1_dim,   #
            out_features=out_seq_len*input_size
            )

        # self.init_weights()

    # def init_weights(self) -> None:
    #     initrange = 0.1
    #     self.encoder_input_layer.weight.data.uniform_(-initrange, initrange)
    #     self.linear_mapping.bias.data.zero_()
    #     self.linear_mapping.weight.data.uniform_(-initrange, initrange)


        # self.init_weights()

    # def init_weights(self) -> None:
    #     initrange = 0.1
    #     self.encoder.weight.data.uniform_(-initrange, initrange)
    #     self.decoder.bias.data.zero_()
    #     self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, 
                src: Tensor, #input sequence (to encoder)
                tgt: Tensor, #decoder sequence (input to decoder)
                src_mask: Tensor=None,  #input sequence mask (prevents data points from target sequence to be used)
                tgt_mask: Tensor=None) -> Tensor:

        # Pass throguh the input layer right before the encoder

        # print(f'src: {src.shape}')
        # print(f'tgt: {tgt.shape}')

        
        # print(f'encoder_input_layer output: {src.shape}')

        # Pass through the positional encoding layer
        
        # print(f'position encoder output: {src.shape}')

        src = self.encoder_input_layer(src)

        
        src = self.pos_encoder(src)

        # Pass through all the stacked encoder layers in the encoder
        src = self.encoder(src=src)

        
        # print(f'ecoder output: {src.shape}')

        # # Pass decoder input through decoder input layer
        decoder_output = self.decoder_input_layer(tgt)

        # # print(f'decoder input layer output: {tgt.shape}')

        # # Pass throguh decoder
        decoder_output = self.decoder(
            tgt=decoder_output,
            memory=src,
            tgt_mask=tgt_mask,
            memory_mask=src_mask
            )

        # # print(f'decoder output: {decoder_output.shape}')

        # # Pass through the linear mapping layer
        # decoder_output= self.linear_mapping(decoder_output.flatten(start_dim=1))

        linear_output = self.linear_layer1(decoder_output.flatten(start_dim=1))

        linear_output=self.linear_layer2(linear_output)

        preds = linear_output.reshape(src.shape[0], out_seq_len, input_size)

        # print(f'decoder output after linear mapping {decoder_output.shape}')
        # print(f'preds {preds.shape}')

        return preds

# def generate_square_subsequent_mask(sz: int) -> Tensor:
#     """Generates an upper-triangular matrix of -inf, with zeros on diag."""
#     return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [ ]:
def generate_square_subsequent_mask(dim1: int, dim2: int, dim3: int) -> Tensor:
    """
    Generates an upper-triangular matrix of -inf, with zeros on diag.
    Modified from: 
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    Args:
        dim1: int, batch_size * n_heads
        dim2: int. For src and trg masking this must be target sequence length. 
        dim3: int. For src masking, this must be encoder sequence length.
              For trg masking, this must be target sequence length 
    Return:
        A Tensor of shape [dim1, dim2, dim3]
    """
    return torch.triu(torch.ones(dim1, dim2, dim3) * float('-inf'), diagonal=1)

# Input length
# enc_seq_len = 120

# # Output length
# output_sequence_length = 3

# # Heads in attention layers
# n_heads = 8

# batch_size = 32

# # Make src mask for decoder with size:
# # [batch_size*n_heads, output_sequence_length, enc_seq_len]
# src_mask = generate_square_subsequent_mask(
#     dim1=batch_size*n_heads,
#     dim2=output_sequence_length,
#     dim3=enc_seq_len
#     )

# # Make tgt mask for decoder with size:
# # [batch_size*n_heads, output_sequence_length, output_sequence_length]
# tgt_mask = generate_square_subsequent_mask( 
#     dim1=batch_size*n_heads,
#     dim2=output_sequence_length,
#     dim3=output_sequence_length
#     )

In [ ]:
def train_transformer(model, train_dataloader, val_dataloader, num_epochs, learning_rate, device):
    loss_function = nn.MSELoss(reduction='mean')
    optimiser = torch.optim.Adam(model.parameters(), lr = learning_rate)
    # scheduler = torch.optim.lr_scheduler.StepLR(optimiser, 3.0, gamma=0.95)


    # train_loss, val_loss = np.zeros(num_epochs), np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    # n_heads=4

    for epoch in tqdm(range(num_epochs)):
        # Loop over batch values 
        runningLoss_train = 0. 
        # print(f'Learning rate: {scheduler.get_last_lr()[0]}')

        for idx, (batch_inputs, batch_targets) in enumerate(train_dataloader):
            
            # print(f'batch_inputs shape training set:{batch_inputs.shape[0]}')
            
            # Save batch on GPU
            # print(f'batch_targets: {batch_targets.shape}')
            batch_decoder_inputs = batch_inputs[:,-3:,:]
            batch_inputs, batch_targets, batch_decoder_inputs = batch_inputs.to(device), batch_targets.to(device), batch_decoder_inputs.to(device)

            # batch_inputs, batch_targets = batch_inputs.to(device), batch_targets.to(device)
            # print(f'batch inputs shape: {batch_inputs.shape}')

            # src_mask = generate_square_subsequent_mask(
            #     dim1=batch_inputs.shape[0]*n_heads,
            #     dim2=batch_decoder_inputs.shape[1],
            #     dim3=batch_inputs.shape[1] #encoder seq_length
            #     ).to(device)
            
            # # print(f'src_mask.shape: {src_mask.shape}')

            # tgt_mask = generate_square_subsequent_mask( 
            #     dim1=batch_inputs.shape[0]*n_heads,
            #     dim2=batch_decoder_inputs.shape[1],
            #     dim3=batch_decoder_inputs.shape[1]
            #     ).to(device)
            # print(f'tgt_mask.shape: {tgt_mask.shape}')

            # Means we are training the model, so uses techniques such as dropout etc., otherwise model.train(model=False)
            model.train() 

            #set gradients to zero
            optimiser.zero_grad()
            preds = model(batch_inputs, batch_decoder_inputs)
            # print(f'shape of training predictions: {preds.shape}')
            # print(f'shape of targets: {batch_targets.shape}')
        
            loss = loss_function(preds, batch_targets)
            
            

            loss.backward()
            optimiser.step()
            runningLoss_train += loss.item()

        train_loss[epoch] = runningLoss_train / len(train_dataloader)

        print(f"Epoch: [{epoch + 1}/{num_epochs}]", f"Training loss: {runningLoss_train/len(train_dataloader)}")

        # Evaluate on validation set

        model.eval() # means we are evaluating the model, stops process such as dropout etc. 
        runningLoss_val = 0.

        with torch.no_grad(): # makes sure gradient is not stored 
            for idx, (batch_inputs, batch_targets) in enumerate(val_dataloader):

            #     src_mask = generate_square_subsequent_mask(
            #     dim1=batch_inputs.shape[0]*n_heads,
            #     dim2=batch_decoder_inputs.shape[1],
            #     dim3=batch_inputs.shape[1] #encoder seq_length
            #     ).to(device)
            
            # # print(f'src_mask.shape: {src_mask.shape}')

            #     tgt_mask = generate_square_subsequent_mask( 
            #         dim1=batch_inputs.shape[0]*n_heads,
            #         dim2=batch_decoder_inputs.shape[1],
            #         dim3=batch_decoder_inputs.shape[1]
            #         ).to(device)


            #     batch_decoder_inputs = batch_inputs[:,-50:,:]                
                # batch_inputs, batch_targets= batch_inputs.to(device), batch_targets.to(device)
                # batch_inputs, batch_targets, batch_decoder_inputs = batch_inputs.to(device), batch_targets.to(device), batch_decoder_inputs.to(device)

                batch_decoder_inputs = batch_inputs[:,-3:,:]
                batch_inputs, batch_targets, batch_decoder_inputs = batch_inputs.to(device), batch_targets.to(device), batch_decoder_inputs.to(device)

                # src_mask = model.generate_square_subsequent_mask(batch_inputs.shape[1]).to(device)
                # tgt_mask = model.generate_square_subsequent_mask(batch_decoder_inputs.shape[1]).to(device)
                
                optimiser.zero_grad() #WHY?
                preds = model(batch_inputs, batch_decoder_inputs)
                loss = loss_function(preds, batch_targets)
                runningLoss_val += loss.item()

        val_loss[epoch] = runningLoss_val/len(val_dataloader)

        print(f"Epoch: [{epoch + 1}/{num_epochs}]", f"Validation loss: {runningLoss_val/len(val_dataloader)}")

        # Additional information
        EPOCH = epoch
        PATH = r"D:\Study 2 Results and Models\Study 2 Model Checkpoints" + '\\' + 'model.pt'
        LOSS_VAL = runningLoss_val/len(val_dataloader)
        LOSS_TRAIN = runningLoss_train/len(train_dataloader)

        torch.save({
                    'epoch': EPOCH,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimiser.state_dict(),
                    'loss_train': LOSS_TRAIN,
                    'loss_val': LOSS_VAL,
                    }, PATH)

        # scheduler.step()

    return train_loss, val_loss

In [ ]:
out_seq_len=output_window
in_seq_len=input_window
input_size=len(features)

transformer_model = TransformerModel(
    input_size=input_size,
    in_seq_len = in_seq_len,  
    max_seq_len= in_seq_len,
    out_seq_len = out_seq_len,
    d_model =60, # aka d_model 
    n_encoder_layers=2,
    n_decoder_layers=2,
    n_heads=4,
    dropout_encoder=0.2, 
    dropout_decoder=0.2,
    dim_feedforward_encoder=200, #2048 is default value
    linear_layer1_dim=200,
    dim_feedforward_decoder=200

    ).to(DEVICE)

num_epochs=50
learning_rate=0.001

train_history, val_history = train_transformer(transformer_model, train_dataloader=train_dataloader, val_dataloader=val_dataloader, num_epochs=num_epochs, learning_rate=learning_rate, device=DEVICE) #ADJUSTED

In [ ]:
6/4

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)



In [ ]:
count_parameters(transformer_model)

output = transformer (
    src=src, 
    tgt=trg,
    src_mask=src_mask,
    tgt_mask=tgt_mask
    )

In [ ]:

# def get_src_trg(
#     sequence: torch.Tensor, 
#     enc_seq_len: int, 
#     dec_seq_len: int, 
#     target_seq_len: int
#     ) -> Tuple[torch.tensor, torch.tensor, torch.tensor]:

#     """
#     Generate the src (encoder input), trg (decoder input) and trg_y (the target)
#     sequences from a sequence. 
#     Args:
#         sequence: tensor, a 1D tensor of length n where 
#                 n = encoder input length + target sequence length  
#         enc_seq_len: int, the desired length of the input to the transformer encoder
#         target_seq_len: int, the desired length of the target sequence (the 
#                         one against which the model output is compared)
#     Return: 
#         src: tensor, 1D, used as input to the transformer model
#         trg: tensor, 1D, used as input to the transformer model
#         trg_y: tensor, 1D, the target sequence against which the model output
#             is compared when computing loss. 
#     """
#     assert len(sequence) == enc_seq_len + target_seq_len, "Sequence length does not equal (input length + target length)"

#     # encoder input
#     src = sequence[:enc_seq_len] 

#     # decoder input. As per the paper, it must have the same dimension as the 
#     # target sequence, and it must contain the last value of src, and all
#     # values of trg_y except the last (i.e. it must be shifted right by 1)
#     trg = sequence[enc_seq_len-1:len(sequence)-1]

#     assert len(trg) == target_seq_len, "Length of trg does not match target sequence length"

#     # The target sequence against which the model output will be compared to compute loss
#     trg_y = sequence[-target_seq_len:]

#     assert len(trg_y) == target_seq_len, "Length of trg_y does not match target sequence length"

#     return src, trg, trg_y.squeeze(-1) # change size from [batch_size, target_seq_len, num_features] to [batch_size, target_seq_len] 


In [ ]:

# import torch

# # Create some dummy data - a univariate time series of random numbers
# dummy_data = torch.rand(1000)

# # Define some hyperparameters
# enc_seq_len = 120 # length of input given to encoder
# dec_seq_len = 12 # length of input given to decoder
# target_seq_len = 3 # the desired length of the model forecast

# # Make a sub-sequence from the dummy data
# sub_sequence = dummy_data[0:enc_seq_len+target_seq_len]

# # Get src, trg, trg_y
# src, trg, trg_y = get_src_trg(
#     sequence=sub_sequence,
#     enc_seq_len=enc_seq_len,
#     dec_seq_len=dec_seq_len,
#     target_seq_len=target_seq_len
#     )

In [ ]:
# enc_seq_len = 120 # length of input given to encoder
# dec_seq_len = 12 # length of input given to decoder
# target_seq_len = 3 # the desired length of the model forecast


In [ ]:
# # ntokens = len(vocab)  # size of vocabulary
# d_model = 512  # embedding dimension
# d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
# nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
# nhead = 2  # number of heads in nn.MultiheadAttention
# dropout = 0.2  # dropout probability
# in_seq_len=input_window
# out_seq_len=output_window
# output_size=len(features)


# model = TransformerModel(d_model, nhead, d_hid, nlayers, dropout, in_seq_len, out_seq_len, output_size).to(DEVICE)

In [ ]:
# from torch.utils.data import DataLoader, Dataset

# class windowDataset(Dataset):
#     def __init__(self, y, input_window=80, output_window=20, stride=5):
#         #총 데이터의 개수
#         L = y.shape[0]
#         #stride씩 움직일 때 생기는 총 sample의 개수
#         num_samples = (L - input_window - output_window) // stride + 1

#         #input과 output
#         X = np.zeros([input_window, num_samples])
#         Y = np.zeros([output_window, num_samples])

#         for i in np.arange(num_samples):
#             start_x = stride*i
#             end_x = start_x + input_window
#             X[:,i] = y[start_x:end_x]

#             start_y = stride*i + input_window
#             end_y = start_y + output_window
#             Y[:,i] = y[start_y:end_y]

#         X = X.reshape(X.shape[0], X.shape[1], 1).transpose((1,0,2))
#         Y = Y.reshape(Y.shape[0], Y.shape[1], 1).transpose((1,0,2))
#         self.x = X
#         self.y = Y
        
#         self.len = len(X)
#     def __getitem__(self, i):
#         return self.x[i], self.y[i, :-1], self.y[i,1:]
#     def __len__(self):
#         return self.len

In [ ]:
# from torch.nn import Transformer
# from torch import nn
# import torch
# import math

# class TFModel(nn.Module):
#     def __init__(self,d_model, nhead, nhid, nlayers, dropout=0.5):
#         super(TFModel, self).__init__()
#         self.transformer = Transformer(d_model=d_model, nhead=nhead, dim_feedforward=nhid, num_encoder_layers=nlayers, num_decoder_layers=nlayers,dropout=dropout)
#         self.pos_encoder = PositionalEncoding(d_model, dropout)
#         self.pos_encoder_d = PositionalEncoding(d_model, dropout)
#         self.linear = nn.Linear(d_model, 1)
#         self.encoder = nn.Linear(1, d_model)
#         self.encoder_d = nn.Linear(1, d_model)

#     def generate_square_subsequent_mask(self, sz):
#         mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
#         mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#         return mask

#     def forward(self, src, tgt, srcmask, tgtmask):
#         src = self.encoder(src)
#         src = self.pos_encoder(src)

#         tgt = self.encoder_d(tgt)
#         tgt = self.pos_encoder_d(tgt)
#         output = self.transformer(src.transpose(0,1), tgt.transpose(0,1), srcmask, tgtmask)
#         output = self.linear(output)
#         return output


# def gen_attention_mask(x):
#     mask = torch.eq(x, 0)
#     return mask

In [ ]:
#OLD VERSION
def train_transformer(model, train_dataloader, val_dataloader, num_epochs, learning_rate, device):
    loss_function = nn.MSELoss(reduction='mean')
    optimiser = torch.optim.Adam(model.parameters(), lr = learning_rate)

    # train_loss, val_loss = np.zeros(num_epochs), np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    n_heads=4

    for epoch in tqdm(range(num_epochs)):
        # Loop over batch values 
        runningLoss_train = 0. 

        for idx, (batch_inputs, batch_targets) in enumerate(train_dataloader):

            # print(f'batch_inputs shape training set:{batch_inputs.shape[0]}')
            
            # Save batch on GPU
            # print(f'batch_targets: {batch_targets.shape}')
            batch_decoder_inputs = batch_inputs[:,-50:,:]
            batch_inputs, batch_targets, batch_decoder_inputs = batch_inputs.to(device), batch_targets.to(device), batch_decoder_inputs.to(device)

            # print(f'batch inputs shape: {batch_inputs.shape}')

            src_mask = generate_square_subsequent_mask(
                dim1=batch_inputs.shape[0]*n_heads,
                dim2=batch_decoder_inputs.shape[1],
                dim3=batch_inputs.shape[1] #encoder seq_length
                ).to(device)
            
            # print(f'src_mask.shape: {src_mask.shape}')

            tgt_mask = generate_square_subsequent_mask( 
                dim1=batch_inputs.shape[0]*n_heads,
                dim2=batch_decoder_inputs.shape[1],
                dim3=batch_decoder_inputs.shape[1]
                ).to(device)
            # print(f'tgt_mask.shape: {tgt_mask.shape}')

            # Means we are training the model, so uses techniques such as dropout etc., otherwise model.train(model=False)
            model.train() 

            #set gradients to zero
            optimiser.zero_grad()
            preds = model(batch_inputs, batch_decoder_inputs, src_mask = src_mask, 
                        tgt_mask=tgt_mask)
            # print(f'shape of training predictions: {preds.shape}')
            # print(f'shape of targets: {batch_targets.shape}')
        
            loss = loss_function(preds, batch_targets)
            

            loss.backward()
            optimiser.step()
            runningLoss_train += loss.item()

        train_loss[epoch] = runningLoss_train / len(train_dataloader)

        print(f"Epoch: [{epoch + 1}/{num_epochs}]", f"Training loss: {runningLoss_train/len(train_dataloader)}")

        # Evaluate on validation set

        model.eval() # means we are evaluating the model, stops process such as dropout etc. 
        runningLoss_val = 0.

        with torch.no_grad(): # makes sure gradient is not stored 
            for idx, (batch_inputs, batch_targets) in enumerate(val_dataloader):

                src_mask = generate_square_subsequent_mask(
                dim1=batch_inputs.shape[0]*n_heads,
                dim2=batch_decoder_inputs.shape[1],
                dim3=batch_inputs.shape[1] #encoder seq_length
                ).to(device)
            
            # print(f'src_mask.shape: {src_mask.shape}')

                tgt_mask = generate_square_subsequent_mask( 
                    dim1=batch_inputs.shape[0]*n_heads,
                    dim2=batch_decoder_inputs.shape[1],
                    dim3=batch_decoder_inputs.shape[1]
                    ).to(device)


                batch_decoder_inputs = batch_inputs[:,-50:,:]                
                batch_inputs, batch_targets, batch_decoder_inputs = batch_inputs.to(device), batch_targets.to(device), batch_decoder_inputs.to(device)
                # src_mask = model.generate_square_subsequent_mask(batch_inputs.shape[1]).to(device)
                # tgt_mask = model.generate_square_subsequent_mask(batch_decoder_inputs.shape[1]).to(device)
                
                optimiser.zero_grad() #WHY?
                preds = model(batch_inputs, batch_decoder_inputs, src_mask = src_mask, 
                        tgt_mask=tgt_mask)
                loss = loss_function(preds, batch_targets)
                runningLoss_val += loss.item()

        val_loss[epoch] = runningLoss_val/len(val_dataloader)

        print(f"Epoch: [{epoch + 1}/{num_epochs}]", f"Validation loss: {runningLoss_val/len(val_dataloader)}")

    return train_loss, val_loss

In [ ]:
def test_transformer(model, dataloader, device):
    loss_function = nn.MSELoss(reduction='mean')
    model.eval()
    actual_output, pred_output = [], []
    running_loss = 0. 
    n_heads =4
    with torch.no_grad():
        for idx, (batch_inputs, batch_targets) in tqdm(enumerate(dataloader)):

            # batch_decoder_inputs = batch_inputs[:,-50:,:]
            batch_decoder_inputs = batch_inputs[:,-3:,:]
            batch_inputs, batch_targets, batch_decoder_inputs = batch_inputs.to(device), batch_targets.to(device), batch_decoder_inputs.to(device)
            # src_mask = generate_square_subsequent_mask(
            #         dim1=batch_inputs.shape[0]*n_heads,
            #         dim2=batch_decoder_inputs.shape[1],
            #         dim3=batch_inputs.shape[1] #encoder seq_length
            #         ).to(device)
                
            # # print(f'src_mask.shape: {src_mask.shape}')

            # tgt_mask = generate_square_subsequent_mask( 
            #         dim1=batch_inputs.shape[0]*n_heads,
            #         dim2=batch_decoder_inputs.shape[1],
            #         dim3=batch_decoder_inputs.shape[1]
            #         ).to(device)

            # if idx==0:
            #     batch_preds = model(batch_inputs)
            #     # print(f'batch shape: {batch_preds.shape}')
            #     loss = loss_function(batch_preds, batch_targets)
            #     running_loss += loss.item()
            #     current_preds = batch_preds
            #     all_preds = batch_preds

            # else:
            #     batch_preds = model(batch_inputs)
            #     print(f'batch shape: {batch_preds.shape}')
            #     loss = loss_function(batch_preds, batch_targets)
            #     running_loss += loss.item()
            #     all_preds = torch.cat((current_preds, batch_preds), dim=0)
            #     current_preds = batch_preds

            batch_preds = model(batch_inputs, batch_decoder_inputs)
                        
            loss = loss_function(batch_preds, batch_targets)
            running_loss += loss.item()
            actual_output.append(batch_targets)
            pred_output.append(batch_preds)


            #             lst = []
            # print(f'{x.size()}')
            # for i in range(10):
            #     x += i  # say we do something with x at iteration i
            #     lst.append(x)
            # # lstt = torch.stack([x for _ in range(10)])
            # lstt = torch.stack(lst)
            # print(lstt.size())

        total_loss = running_loss / len(dataloader)

        actual_output_tensor = torch.vstack(actual_output)
        pred_output_tensor = torch.vstack(pred_output)
    
    return pred_output_tensor, actual_output_tensor, total_loss

In [ ]:
# epoch = 2000
# from tqdm import tqdm

# model.train()
# progress = tqdm(range(epoch))
# for i in progress:
#     batchloss = 0.0
    
#     for (inputs, dec_inputs, outputs) in DataLoader:
#         optimizer.zero_grad()
#         src_mask = model.generate_square_subsequent_mask(inputs.shape[1]).to(device)
#         tgt_mask = model.generate_square_subsequent_mask(dec_inputs.shape[1]).to(device)

#         result = model(inputs.float().to(device), dec_inputs.float().to(device), src_mask, tgt_mask)
#         loss = criterion(result.permute(1,0,2), outputs.float().to(device))
        
#         loss.backward()
#         optimizer.step()
#         batchloss += loss
#     progress.set_description("{:0.5f}".format(batchloss.cpu().item() / len(train_loader)))

In [ ]:
# # # # Train LSTM 
# input_size=len(features)
# hidden_size=128
# num_layers=4
# in_seq_len=input_window
# out_seq_len=output_window
# output_size=len(features)
# num_epochs=4
# learning_rate= 0.001


# model_LSTM = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, in_seq_len=in_seq_len, out_seq_len=out_seq_len, output_size=output_size, device=DEVICE).to(DEVICE)

# train_history, val_history = train_LSTM(model_LSTM, train_dataloader=train_dataloader, val_dataloader=val_dataloader, num_epochs=num_epochs, learning_rate=learning_rate, device=DEVICE) #ADJUSTED

In [ ]:
plt.plot(train_history[2:], label = 'Training loss')
plt.plot(val_history[2:], label = 'Val loss')
plt.xlabel('Iterations')
plt.ylabel('MSE Loss')
plt.title('Training and Validation MSE Loss')
plt.legend()
plt.show()

In [ ]:
pred_val_output, actual_val_output, val_loss = test_transformer(transformer_model, val_dataloader, DEVICE)

predicted_values_val = pred_val_output.cpu().numpy()
actual_values_val = actual_val_output.cpu().numpy()

print(f'val loss: {val_loss}')
print(f'Shape of predicted values test: {predicted_values_val.shape}')
print(f'shape of actual values test: {actual_values_val.shape}')

predicted_val_denorm = denormalise(predicted_values_val, scalars)
actual_val_denorm = denormalise(actual_values_val, scalars)

print(f'Shape of predicted values test post denormalisation: {predicted_val_denorm.shape}')
print(f'shape of actual values test post denormalisation: {actual_val_denorm.shape}')

In [ ]:
val_mse_loss, val_mse_std = mse_loss(predicted_val_denorm, actual_val_denorm, reduction='mean', format='np')
val_mae_loss, val_mae_std = mae_loss(predicted_val_denorm, actual_val_denorm, reduction='mean', format='np')

print(f'Val MSE Loss: {val_mse_loss}')
print(f'Val MSE std: {val_mse_std}')
print(f'Val MAE Loss: {val_mae_loss}')
print(f'Val MAE std: {val_mae_std}')

In [ ]:
10000 ** (2*1/512)

### Benchmark

#### Benchmark against naive method 

In [ ]:
## benchmark against naive output (making prediction only last value)
naive_1_train_output = np.zeros(Y_train_data.shape)

for f in range(len(features)):
    for s in range(Y_train_data.shape[0]):
        naive_1_train_output[s,:,f] = np.ones((output_window)) * X_train_data[s,-1,f]



# print(naive_1_test_output.shape)


# naive_1_test_output[1,:,1]

# X_test_data[1,:,1]



In [ ]:
naive_1_mse_loss, naive_1_mse_std = mse_loss(naive_1_train_output, Y_train_data, reduction='mean', format='np')
naive_1_mae_loss, naive_1_mae_std = mae_loss(naive_1_train_output, Y_train_data, reduction='mean', format='np')

print(f'naive_1 MSE Loss: {naive_1_mse_loss}')
print(f'naive_1 MSE std: {naive_1_mse_std} ')
print(f'naive_1 MAE Loss: {naive_1_mae_loss}')
print(f'naive_1 MAE std: {naive_1_mae_std}')

In [ ]:
## benchmark against mean output

naive_average_train_output = np.zeros(Y_train_data.shape)

for f in range(len(features)):
    for s in range(Y_train_data.shape[0]):
        naive_average_train_output[s,:,f] = np.ones((output_window)) * np.mean(X_train_data[s,:,f])



print(naive_average_train_output.shape)



In [ ]:
naive_average_mse_loss, naive_average_mse_std = mse_loss(naive_average_train_output, Y_train_data, reduction='mean', format='np')
naive_average_mae_loss, naive_average_mae_std = mae_loss(naive_average_train_output, Y_train_data, reduction='mean', format='np')

print(f'average MSE Loss: {naive_average_mse_loss}')
print(f'average MSE std: {naive_average_mse_std} ')
print(f'average MAE Loss: {naive_average_mae_loss}')
print(f'average MAE std: {naive_average_mae_std}')

### Save Progress and Results

In [ ]:
model = 'Trasformer'
exp_ID = '016'

out_fname = r'D:\Study 2 Results and Models\Investigation Results' + '\\' + 'Study 2 Experimentation Results' + '.txt'


In [ ]:
# if os.path.exists(out_fname):
#     print('ERROR: File with this name already exists, try alternative name.')
    
# else:
with open(out_fname, 'a+') as text_file:
    
    print('Experiment Report', file=text_file)
    print(f'{exp_ID}: {date.today()}',file=text_file)
    # print('\n', file=text_file)
    print(f'model: {model}', file=text_file)
    print(f'features: {features} \n', file=text_file)

    print('Information on Data', file=text_file)
    print(f'train size: {train_sample_sum}', file=text_file)
    print(f'validation size: {val_sample_sum}', file=text_file)
    print(f'Train subjects: {train_subjects}', file=text_file)
    print(f'Val subjects: {val_subjects}', file=text_file)
    print(f'Scalars: {scalars} \n', file=text_file)

    print('Model Hyperparameters', file=text_file)
    print(f'input window size: {input_window}', file=text_file)
    print(f'output window size: {output_window}', file=text_file)
    print(f'stride: {stride}', file=text_file)
    print(f'epochs: {num_epochs}', file=text_file)
    # print(f'hidden size: {hidden_size}', file=text_file)
    # print(f'number of layers: {num_layers}', file=text_file)
    print(f'learning rate: {learning_rate}', file=text_file)
    print(f'optimiser: MSE', file=text_file)

    print('\n', file = text_file)
    print(f'Model Architecture', file=text_file)
    print(transformer_model, file=text_file)

    print(f'Losses during training:', file=text_file)
    print(f'training loss: {train_history}', file=text_file)
    print(f'validation history: {val_history}', file=text_file)

    print('\n', file=text_file)
    print(f'Denomralised MSE and MAE: ', file=text_file)
    print(f'Validation MSE and std(degrees): {val_mse_loss:.3f}, {val_mse_std:.3f}', file=text_file)
    print(f'Validation MAE and std (degrees): {val_mae_loss:.3f}, {val_mae_std:.3f}', file=text_file)
    # print('\n', file=text_file)
    print('--------------------------------------------------------------------------------------------------------', file=text_file)


print("Generation of text file complete.")

In [ ]:
plt_path = r'D:\Study 2 Results and Models\Investigations Plots' + '\\'  + 'Exp' + str(exp_ID) + '-' + str(date.today()) + '-' + model + '-In' + str(input_window) + '-Out' + str(output_window) + '(loss)' + '.png'
fst_e = 2 #first epoch to start plotting for 
x = [i for i in range(fst_e,num_epochs)]
# print(x)
# print(train_history[20:145].shape)
plt.plot(x,train_history[fst_e:], label = 'Training loss')
plt.plot(x,val_history[fst_e:], label = 'Val loss')
# plt.plot(train_history, label = 'Training loss')
# plt.plot(val_history, label = 'Val loss')


plt.xlabel('Iterations')
plt.ylabel('MSE Loss')
plt.title('Train and Val MSE Loss')
plt.legend()
plt.savefig(plt_path)

#### Save model

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in transformer_model.state_dict():
    print(param_tensor, "\t", transformer_model.state_dict()[param_tensor].size())

In [ ]:
# Export model
# export_path = r'C:\Users\Rania\rbk9\Study 1 Results and Models\Study 1 Models' + '\\'  + 'Exp' + str(exp_ID) + '-' + str(date.today()) + '-' + model+ '-In' + str(input_window) + '-Out' + str(output_window) +'.pickle'
export_path = r'D:\Study 2 Results and Models\Study 2 Model' + '\\'  + 'Exp' + str(exp_ID) + '-' + str(date.today()) + '-' + model+ '-In' + str(input_window) + '-Out' + str(output_window) +'.pickle'

# export_path = r'C:\Users\Rania\rbk9\Study 1 Results and Models\Study 1 Models\LSTM_4' + '\\'  + 'Exp' + str(exp_ID) + '-' + str(date.today()) + '-' + model+ '-In' + str(input_window) + '-Out' + str(output_window) +'.pickle'


export_path

In [ ]:
torch.save(transformer_model.state_dict(), export_path)

In [ ]:
import_path = r'D:\Study 1 Results and Models\Study 1 Models\LSTM_4' + '\\' + 'Exp014-2022-02-10-LSTM-In120-Out3.pickle'
# # # import_path = r'C:\Users\Rania\rbk9\Study 1 Results and Models\Study 1 Models\CNN_1' + '\\' + 'Exp001-2022-01-24-CNN-In96-Out1.pickle'
# # # model_CNN = CNN(input_size, output_size=output_size, kernel_size=kernel_size, stride=1, in_seq_len=in_seq_len, out_seq_len=out_seq_len,  device = DEVICE).to(DEVICE)

input_size=len(features)
hidden_size=128
num_layers=4
in_seq_len=input_window
out_seq_len=output_window
output_size=len(features)
# num_epochs=60

# # learning_rate= 0.001

model_LSTM = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, in_seq_len=in_seq_len, out_seq_len=out_seq_len, output_size=output_size, device=DEVICE).to(DEVICE)
model_LSTM.load_state_dict(torch.load(import_path))
model_LSTM.eval()

In [ ]:
pred_val_output, actual_val_output, val_loss = test_LSTM(model_LSTM, val_dataloader, DEVICE)

predicted_values_val = pred_val_output.cpu().numpy()
actual_values_val = actual_val_output.cpu().numpy()

print(f'val loss: {val_loss}')
print(f'Shape of predicted values test: {predicted_values_val.shape}')
print(f'shape of actual values test: {actual_values_val.shape}')

predicted_val_denorm = denormalise(predicted_values_val, scalars)
actual_val_denorm = denormalise(actual_values_val, scalars)

print(f'Shape of predicted values test post denormalisation: {predicted_val_denorm.shape}')
print(f'shape of actual values test post denormalisation: {actual_val_denorm.shape}')

In [ ]:
val_mse_loss, val_mse_std = mse_loss(predicted_val_denorm, actual_val_denorm, reduction='mean', format='np')
val_mae_loss, val_mae_std = mae_loss(predicted_val_denorm, actual_val_denorm, reduction='mean', format='np')

print(f'Val MSE Loss: {val_mse_loss}')
print(f'Val MSE std: {val_mse_std}')
print(f'Val MAE Loss: {val_mae_loss}')
print(f'Val MAE std: {val_mae_std}')

In [ ]:
# for p in model_LSTM.parameters():
#     print(p.device)

In [ ]:
# # Testing function 
# def test_LSTM(model, dataloader):
#     loss_function = nn.MSELoss(reduction='mean')
#     model.eval()
#     actual_output, pred_output = [], []
#     running_loss = 0. 
    
#     with torch.no_grad():
#         for idx, (batch_inputs, batch_targets) in tqdm(enumerate(dataloader)):

#             batch_inputs, batch_targets = batch_inputs.to(DEVICE), batch_targets.to(DEVICE)

#             # if idx==0:
#             #     batch_preds = model(batch_inputs)
#             #     # print(f'batch shape: {batch_preds.shape}')
#             #     loss = loss_function(batch_preds, batch_targets)
#             #     running_loss += loss.item()
#             #     current_preds = batch_preds
#             #     all_preds = batch_preds

#             # else:
#             #     batch_preds = model(batch_inputs)
#             #     print(f'batch shape: {batch_preds.shape}')
#             #     loss = loss_function(batch_preds, batch_targets)
#             #     running_loss += loss.item()
#             #     all_preds = torch.cat((current_preds, batch_preds), dim=0)
#             #     current_preds = batch_preds

#             batch_preds = model(batch_inputs)
#             # print(f'batch preds: {batch_preds.type}')
#             loss = loss_function(batch_preds, batch_targets)
#             running_loss += loss.item()
#             actual_output.append(batch_targets)
#             pred_output.append(batch_preds)


#             #             lst = []
#             # print(f'{x.size()}')
#             # for i in range(10):
#             #     x += i  # say we do something with x at iteration i
#             #     lst.append(x)
#             # # lstt = torch.stack([x for _ in range(10)])
#             # lstt = torch.stack(lst)
#             # print(lstt.size())

#         total_loss = running_loss / len(dataloader)

#         actual_output_tensor = torch.vstack(actual_output)
#         pred_output_tensor = torch.vstack(pred_output)
    
#     return pred_output_tensor, actual_output_tensor, total_loss

In [ ]:
# # Creates dataset object that gets individual samples for training/testing so that the Dataloader can generate batches
# class gaitDataset(Dataset):
#     def __init__(self, x, y):
#         self.x = x 
#         self.y = y 

#     def __len__(self):
#         return self.x.shape[0]
    
#     def __getitem__(self, index):
#         X_sample = self.x[index, :, :]
#         Y_sample = self.y[index, :, :]
#         return X_sample, Y_sample

In [ ]:
# # X_test_norm = normalise_transform(X_test_data, scalars)
# # Y_test_norm = normalise_transform(Y_test_data, scalars)

# # Convert to Tensor 
# # do not store on GPU (yet)
# X_test = torch.from_numpy(X_train_norm).float()
# Y_test = torch.from_numpy(Y_train_norm).float()

# print(f'X_test shape: {X_test.shape}')
# print(f'Y_test shape: {Y_test.shape}')

In [ ]:
# test_dataset = gaitDataset(X_test, Y_test)
# # # test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
# test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False) #ADJUSTED

# print(f"Test Dataset length: {len(test_dataset)}")

In [ ]:
# pred_test_output, actual_test_output, test_loss = test_LSTM(model_LSTM, test_dataloader)

# predicted_values_test = pred_test_output.cpu().numpy() # premute again to make the order of the array (samples, windows, features)
# actual_values_test =  actual_test_output.cpu().numpy()

# print(f'test loss: {test_loss}')

# print(f'Shape of predicted values test: {predicted_values_test.shape}')
# print(f'shape of actual values test: {actual_values_test.shape}')

# predicted_test_denorm = denormalise(predicted_values_test, scalars)
# actual_test_denorm =  denormalise(actual_values_test, scalars)

# print(f'Shape of predicted values test post denormalisation: {predicted_test_denorm.shape}')
# print(f'shape of actual values test post denormalisation: {actual_test_denorm.shape}')

In [ ]:
test_mse_loss, test_mse_std = mse_loss(predicted_test_denorm, actual_test_denorm, reduction='mean', format='np')
test_mae_loss, test_mae_std = mae_loss(predicted_test_denorm, actual_test_denorm, reduction='mean', format='np')

print(f'Test MSE Loss: {test_mse_loss}')
print(f'Test MSE std: {test_mse_std} ')

# MDE= mae_loss(actual_test_denorm[:-1,:,:],actual_test_denorm[1:,:,:],reduction='mean', format='np')
MDE= mae_loss(actual_test_denorm[:,:,:],actual_test_denorm[:,:,:],reduction='mean', format='np')

print(f'Test MAE Loss: {test_mae_loss}')
print(f'Test MAE std: {test_mae_std}')

In [ ]:
plt.hist(np.abs(predicted_test_denorm[:,:,1].reshape(-1,1) - actual_test_denorm[:,:,1].reshape(-1,1)))



In [ ]:
actual = np.array([23,36,28,33,44,22,12,10,1])
predicted = np.array([23, 35.5, 28.1, 33, 45, 15, 13, 8, 0])

actual-predicted

In [ ]:
abs_values = np.abs(actual-predicted)

plt.hist(abs_values)

In [ ]:


np.abs(actual-predicted).std()
